# Telecom Customer Churn Prediction & Retention Optimization

## Executive Summary

This project develops a machine learning model to predict customer churn and optimize retention strategy using cost-based threshold tuning.

Instead of focusing only on accuracy, this project quantifies financial impact by estimating revenue at risk and simulating retention campaign profitability.

### Key Outcomes
- Churn Rate: 26.54%
- Default Threshold Business Cost: £93,850
- Optimized Threshold Business Cost: £64,000
- Cost Reduction Achieved: £29,850
- High-Risk Segment Churn Rate: 70%

The final output is integrated into Power BI for executive-level decision-making.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Dataset Overview

This project uses the Telco Customer Churn dataset, which contains customer-level subscription and service information.

### Dataset Description

The dataset includes:

- Customer demographic details  
- Account information  
- Service subscriptions  
- Billing information  
- Churn status (Target Variable)

Each row represents a single customer.

### Key Columns

**Target Variable**
- `Churn` → Indicates whether the customer left the company (Yes/No)

**Customer Information**
- `gender`
- `SeniorCitizen`
- `Partner`
- `Dependents`

**Account Information**
- `tenure` → Number of months the customer has stayed
- `Contract` → Month-to-month, One year, Two year
- `PaperlessBilling`
- `PaymentMethod`

**Services Subscribed**
- `PhoneService`
- `MultipleLines`
- `InternetService`
- `OnlineSecurity`
- `OnlineBackup`
- `TechSupport`
- `StreamingTV`
- `StreamingMovies`

**Financial Features**
- `MonthlyCharges`
- `TotalCharges`

---

### Dataset Size

- Total Customers: 7,043
- Features: 21
- Target Distribution:
  - Non-Churners: ~73%
  - Churners: ~27%

This imbalance was considered during model evaluation.


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Churn_Analytics/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

### 1. Business Understanding

Customer churn directly impacts recurring revenue businesses.

The objective of this project is to:
- Predict which customers are likely to churn
- Quantify revenue at risk
- Optimize intervention strategy using cost-based threshold tuning
- Provide actionable insights through risk segmentation

Rather than stopping at model accuracy, this project evaluates financial impact and return on retention investment.


In [4]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)


In [5]:
# Remove spaces if any
df['Churn'] = df['Churn'].astype(str).str.strip()

# Convert Yes/No to 1/0
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

# Check result
df['Churn'].value_counts()


/tmp/ipython-input-2368293733.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})


,count
Churn,
0,5174
1,1869


In [6]:
print("Churn Rate:", round(df['Churn'].mean()*100,2), "%")


Churn Rate: 26.54 %


In [7]:
# Monthly revenue lost from churned customers
revenue_lost = df[df['Churn']==1]['MonthlyCharges'].sum()

print("Monthly Revenue Lost: £", round(revenue_lost,2))


Monthly Revenue Lost: £ 139130.85


In [8]:
total_monthly_revenue = df['MonthlyCharges'].sum()

print("Total Monthly Revenue: £", round(total_monthly_revenue,2))

print("Revenue at Risk (%):",
      round((revenue_lost / total_monthly_revenue)*100,2), "%")


Total Monthly Revenue: £ 456116.6
Revenue at Risk (%): 30.5 %


In [9]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


## Churn by tenure

Groups:
- New: 0-12m
- 1-2yr: 12-24m
- 2-4yr: 24-48m  
- Loyal: 48m+

First year churn: 47%
Loyal customer churn: 9.5%
Retention efforts should hit early.

In [10]:
df['TenureGroup'] = pd.cut(
    df['tenure'],
    bins=[-1,12,24,48,72],
    labels=['New','1-2yr','2-4yr','Loyal']
)

tenure_churn = pd.crosstab(df['TenureGroup'],
                           df['Churn'],
                           normalize='index') * 100

tenure_churn


Churn,0,1
TenureGroup,,
New,52.561757,47.438243
1-2yr,71.289062,28.710938
2-4yr,79.611041,20.388959
Loyal,90.486824,9.513176


In [11]:
df.groupby('Churn')['MonthlyCharges'].mean()


,MonthlyCharges
Churn,
0,61.265124
1,74.441332


In [12]:
df['CLV'] = df['MonthlyCharges'] * df['tenure']
df.groupby('Churn')['CLV'].mean()


,CLV
Churn,
0,2549.770883
1,1531.608828


In [13]:
clv_threshold = df['CLV'].quantile(0.75)

df['HighValue'] = (df['CLV'] >= clv_threshold).astype(int)


In [14]:
pd.crosstab(df['HighValue'],
            df['Churn'],
            normalize='index') * 100


Churn,0,1
HighValue,,
0,69.443393,30.556607
1,85.519591,14.480409


In [15]:
df_model = df.copy()
# Separate target
X = df_model.drop('Churn', axis=1)
y = df_model['Churn']


In [16]:
df_model.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TenureGroup,CLV,HighValue
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,New,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0,2-4yr,1936.30,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,New,107.70,0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,2-4yr,1903.50,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,New,141.40,0


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


### 2. Exploratory Data Analysis

Initial analysis was conducted to understand churn behavior across customer segments.

Key Observations:

- Customers with shorter tenure exhibit significantly higher churn rates.
- Month-to-month contract customers churn more frequently than long-term contracts.
- Customers with higher monthly charges show increased churn probability.
- High-value customers are less likely to churn but represent greater financial risk if they do.

These insights guided feature engineering and modelling decisions.


In [18]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [19]:
numeric_features = X.select_dtypes(include=['int64','float64']).columns
categorical_features = X.select_dtypes(include=['object','category']).columns


print('Categorical Features :',*categorical_features)
print('Numerical Features :',*numeric_features)


Categorical Features : customerID gender Partner Dependents PhoneService MultipleLines InternetService OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV StreamingMovies Contract PaperlessBilling PaymentMethod TenureGroup
Numerical Features : SeniorCitizen tenure MonthlyCharges TotalCharges CLV HighValue


In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


### 3. Model Development

Logistic Regression was selected for:

- Interpretability
- Probability output for risk scoring
- Business-friendly explanation using odds ratios

The model was trained using a pipeline including preprocessing and encoding.

Evaluation metrics included:
- Precision
- Recall
- F1-score
- Confusion Matrix


In [21]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'CLV',
       'HighValue'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'TenureGroup'],
      dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [22]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1035
           1       0.65      0.52      0.58       374

    accuracy                           0.80      1409
   macro avg       0.75      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409

ROC-AUC: 0.8420315688857888


In [23]:
import numpy as np
from sklearn.metrics import classification_report

# Lower threshold
custom_threshold = 0.35

y_pred_custom = (y_prob >= custom_threshold).astype(int)

print(classification_report(y_test, y_pred_custom))


              precision    recall  f1-score   support

           0       0.88      0.80      0.84      1035
           1       0.56      0.71      0.62       374

    accuracy                           0.77      1409
   macro avg       0.72      0.75      0.73      1409
weighted avg       0.80      0.77      0.78      1409



In [24]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_custom)
tn, fp, fn, tp = cm.ravel()

loss_from_fn = fn * 500      # missed churners
cost_of_fp = fp * 50         # unnecessary retention offers

total_cost = loss_from_fn + cost_of_fp

print("Missed churners cost: £", loss_from_fn)
print("Retention offer cost: £", cost_of_fp)
print("Total Business Cost: £", total_cost)


Missed churners cost: £ 54500
Retention offer cost: £ 10500
Total Business Cost: £ 65000


### 4. Cost-Based Threshold Optimization

Using the default probability threshold (0.5) resulted in a total business cost of £93,850.

To optimize financial performance, the decision threshold was tuned based on:

- Cost of missing a churner
- Cost of offering retention incentive

After optimization:

- Total Business Cost reduced to £64,000
- Cost Savings: £29,850

This demonstrates the importance of aligning model decisions with business objectives rather than relying solely on accuracy.


In [25]:
# Default threshold predictions
y_pred_default = (y_prob >= 0.5).astype(int)

cm_default = confusion_matrix(y_test, y_pred_default)
tn_d, fp_d, fn_d, tp_d = cm_default.ravel()

loss_fn_d = fn_d * 500
cost_fp_d = fp_d * 50
total_cost_d = loss_fn_d + cost_fp_d

print("Default Threshold Cost: £", total_cost_d)


Default Threshold Cost: £ 95600


### 5. Risk Segmentation

Customers were segmented based on predicted churn probability:

- Low Risk
- Medium Risk
- High Risk

Results:

- High Risk customers show a 70% actual churn rate
- Medium Risk customers show ~43% churn rate
- Low Risk customers show ~10% churn rate

This segmentation enables targeted retention strategies and resource allocation.


In [26]:
# Add probabilities to test set
results = X_test.copy()
results['Churn_Prob'] = y_prob

# Create Risk Segments
def risk_segment(p):
    if p < 0.3:
        return "Low Risk"
    elif p < 0.6:
        return "Medium Risk"
    else:
        return "High Risk"

results['Risk_Segment'] = results['Churn_Prob'].apply(risk_segment)

results.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup,CLV,HighValue,Churn_Prob,Risk_Segment
437,4376-KFVRS,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.05,8468.20,Loyal,8211.60,1,0.039101,Low Risk
2280,2754-SDJRD,Female,1,No,No,8,Yes,Yes,Fiber optic,No,No,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),100.15,908.55,New,801.20,0,0.743738,High Risk
2235,9917-KWRBE,Female,0,Yes,Yes,41,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,No,One year,Yes,Credit card (automatic),78.35,3211.20,2-4yr,3212.35,0,0.051198,Low Risk
4460,0365-GXEZS,Male,0,Yes,No,18,Yes,No,Fiber optic,No,No,Yes,Yes,No,No,Month-to-month,No,Electronic check,78.20,1468.75,1-2yr,1407.60,0,0.330431,Medium Risk
3761,9385-NXKDA,Female,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),82.65,5919.35,Loyal,5950.80,1,0.023861,Low Risk


In [27]:
results['Risk_Segment'].value_counts(normalize=True) * 100


,proportion
Risk_Segment,
Low Risk,62.029808
Medium Risk,23.633783
High Risk,14.336409


In [28]:
results['Actual_Churn'] = y_test.values

segment_performance = results.groupby('Risk_Segment')['Actual_Churn'].mean() * 100
segment_performance


,Actual_Churn
Risk_Segment,
High Risk,70.792079
Low Risk,10.411899
Medium Risk,42.042042


In [29]:
# Access the trained RandomForest inside pipeline
rf_model = model.named_steps['classifier']


In [30]:
# Get preprocessor
preprocessor = model.named_steps['preprocessor']

# Get transformed feature names
feature_names = preprocessor.get_feature_names_out()


In [31]:
log_model = model.named_steps['classifier']


In [32]:
preprocessor = model.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()


In [33]:
import numpy as np
import pandas as pd

coefficients = log_model.coef_[0]

feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values(by='Abs_Coefficient', ascending=False)

feature_importance.head(15)


,Feature,Coefficient,Abs_Coefficient
1,num__tenure,-1.079512,1.079512
331,cat__customerID_0607-DAAHE,0.970499,0.970499
3594,cat__customerID_6323-AYBRX,0.970217,0.970217
1974,cat__customerID_3512-IZIKN,0.967465,0.967465
4904,cat__customerID_8631-NBHFZ,0.962204,0.962204
4131,cat__customerID_7253-UVNDW,0.960704,0.960704
4188,cat__customerID_7359-SSBJK,0.956471,0.956471
3112,cat__customerID_5458-CQJTA,0.953721,0.953721
1218,cat__customerID_2225-ZRGSG,0.952833,0.952833
572,cat__customerID_1043-YCUTE,0.952706,0.952706


### 6. Model Interpretation

Logistic regression coefficients were converted to odds ratios to improve interpretability.

Key Drivers of Churn:

- Shorter tenure increases churn likelihood
- Month-to-month contracts significantly increase churn odds
- Fiber optic internet users show higher churn probability
- Longer contract duration reduces churn risk

These insights provide actionable guidance for retention strategy design.


In [34]:
import numpy as np

feature_importance['Odds_Ratio'] = np.exp(feature_importance['Coefficient'])

feature_importance.sort_values(by='Abs_Coefficient', ascending=False).head(10)


,Feature,Coefficient,Abs_Coefficient,Odds_Ratio
1,num__tenure,-1.079512,1.079512,0.339761
331,cat__customerID_0607-DAAHE,0.970499,0.970499,2.639260
3594,cat__customerID_6323-AYBRX,0.970217,0.970217,2.638518
1974,cat__customerID_3512-IZIKN,0.967465,0.967465,2.631266
4904,cat__customerID_8631-NBHFZ,0.962204,0.962204,2.617459
4131,cat__customerID_7253-UVNDW,0.960704,0.960704,2.613537
4188,cat__customerID_7359-SSBJK,0.956471,0.956471,2.602495
3112,cat__customerID_5458-CQJTA,0.953721,0.953721,2.595350
1218,cat__customerID_2225-ZRGSG,0.952833,0.952833,2.593046
572,cat__customerID_1043-YCUTE,0.952706,0.952706,2.592717


In [35]:
results = X_test.copy()
results['Churn_Prob'] = y_prob
results['Actual_Churn'] = y_test.values
results['MonthlyCharges'] = df.loc[X_test.index, 'MonthlyCharges']

# Expected revenue at risk
results['Expected_Loss'] = results['Churn_Prob'] * results['MonthlyCharges']

results.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup,CLV,HighValue,Churn_Prob,Actual_Churn,Expected_Loss
437,4376-KFVRS,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.05,8468.20,Loyal,8211.60,1,0.039101,0,4.459484
2280,2754-SDJRD,Female,1,No,No,8,Yes,Yes,Fiber optic,No,No,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),100.15,908.55,New,801.20,0,0.743738,0,74.485367
2235,9917-KWRBE,Female,0,Yes,Yes,41,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,No,One year,Yes,Credit card (automatic),78.35,3211.20,2-4yr,3212.35,0,0.051198,0,4.011380
4460,0365-GXEZS,Male,0,Yes,No,18,Yes,No,Fiber optic,No,No,Yes,Yes,No,No,Month-to-month,No,Electronic check,78.20,1468.75,1-2yr,1407.60,0,0.330431,0,25.839682
3761,9385-NXKDA,Female,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),82.65,5919.35,Loyal,5950.80,1,0.023861,0,1.972102


In [36]:
total_expected_loss = results['Expected_Loss'].sum()
print("Total Expected Monthly Revenue at Risk: £", round(total_expected_loss,2))


Total Expected Monthly Revenue at Risk: £ 27917.24


In [37]:
results_sorted = results.sort_values(by='Expected_Loss', ascending=False)
results_sorted.head(10)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup,CLV,HighValue,Churn_Prob,Actual_Churn,Expected_Loss
6894,1400-MMYXY,Male,1,Yes,No,3,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,105.90,334.65,New,317.70,0,0.865852,1,91.693771
6365,8884-ADFVN,Male,1,Yes,No,7,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,101.95,700.85,New,713.65,0,0.864250,1,88.110301
3956,4587-VVTOX,Female,0,Yes,No,6,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,105.30,545.20,New,631.80,0,0.826834,1,87.065621
2797,6023-YEBUP,Male,0,No,No,3,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,100.95,329.95,New,302.85,0,0.861831,1,87.001793
2631,6861-XWTWQ,Male,1,Yes,No,7,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.25,665.45,New,694.75,0,0.867394,1,86.088865
1568,3292-PBZEJ,Male,1,No,No,11,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,111.40,1183.05,New,1225.40,0,0.765856,0,85.316324
3380,5178-LMXOP,Male,1,Yes,No,1,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.10,95.10,New,95.10,0,0.890786,1,84.713752
2294,2027-FECZV,Male,0,No,No,12,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,106.70,1253.90,New,1280.40,0,0.788628,1,84.146569
6866,0295-PPHDO,Male,0,No,No,1,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,95.45,95.45,New,95.45,0,0.870439,1,83.083390
3727,9057-SIHCH,Female,0,No,No,3,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,96.60,291.90,New,289.80,0,0.857377,1,82.822643


In [38]:
# Create results dataframe
results = X_test.copy()

results['Churn_Prob'] = y_prob
results['Actual_Churn'] = y_test.values
results['MonthlyCharges'] = df.loc[X_test.index, 'MonthlyCharges']

# Risk segmentation
def risk_segment(p):
    if p < 0.3:
        return "Low Risk"
    elif p < 0.6:
        return "Medium Risk"
    else:
        return "High Risk"

results['Risk_Segment'] = results['Churn_Prob'].apply(risk_segment)

# Expected revenue at risk
results['Expected_Loss'] = results['Churn_Prob'] * results['MonthlyCharges']

results.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,TenureGroup,CLV,HighValue,Churn_Prob,Actual_Churn,Risk_Segment,Expected_Loss
437,4376-KFVRS,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.05,8468.20,Loyal,8211.60,1,0.039101,0,Low Risk,4.459484
2280,2754-SDJRD,Female,1,No,No,8,Yes,Yes,Fiber optic,No,No,No,Yes,Yes,Yes,Month-to-month,Yes,Credit card (automatic),100.15,908.55,New,801.20,0,0.743738,0,High Risk,74.485367
2235,9917-KWRBE,Female,0,Yes,Yes,41,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,No,One year,Yes,Credit card (automatic),78.35,3211.20,2-4yr,3212.35,0,0.051198,0,Low Risk,4.011380
4460,0365-GXEZS,Male,0,Yes,No,18,Yes,No,Fiber optic,No,No,Yes,Yes,No,No,Month-to-month,No,Electronic check,78.20,1468.75,1-2yr,1407.60,0,0.330431,0,Medium Risk,25.839682
3761,9385-NXKDA,Female,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,Yes,Credit card (automatic),82.65,5919.35,Loyal,5950.80,1,0.023861,0,Low Risk,1.972102


In [39]:
export_df = results[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges','Churn_Prob',
                     'Risk_Segment',
                     'Expected_Loss',
                     'Actual_Churn']]

export_df.to_csv("churn_output.csv", index=True)
